# Problem Set 0: Introduction to training neural networks

<a name='section_1'></a>
<h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #1f77b4">1. Setting up a training dataset</h2>

Okay, now that we've covered some data science basics, let's play with some (fake) data!

<a name='section_2'></a>
<h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #1f77b4">2. Defining a neural network model</h2>

<a name='section_3'></a>
<h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #1f77b4">3. Training</h2>

Let's play with tracking our model training using `wandb`.

https://docs.wandb.ai/guides/track/jupyter/ 

In [22]:
%pip install wandb -qqq

Note: you may need to restart the kernel to use updated packages.


In [25]:
import wandb
wandb.login()

True

In [ ]:
wandb.init(
    project="phys-805-test",
    config={
        "batch_size": 128,
        "learning_rate": 0.01,
        "dataset": "gaussians",
    },
)

<a name='section_4'></a>
<h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #1f77b4">4. Evaluating model performance</h2>

In [ ]:
- define a baseline
- evaluate on the test set

(After this... UDL notebooks?)